# OPM Federal Workforce Fact Data

- Exploratory analysis of March 2024 (most recent available to public) OPM data
- Facts about the federal workforce

In [1]:
import pandas as pd
import getpass
import matplotlib.pyplot as plt
import os

In [3]:
work_directory = '{0}'.format(getpass.getpass('Paste work directory path and then press enter: '))

Paste work directory path and then press enter: ········


In [6]:
os.chdir(work_directory)

In [8]:
print(os.listdir())

['DTgsegrd.txt', 'FS_Employment_Mar2024_Documentation.pdf', 'DTwrksch.txt', 'FACTDATA_MAR2024.TXT', 'DTagy.txt', 'DTppgrd.txt', 'DTpatco.txt', 'DTdate.txt', 'DTagelvl.txt', 'DTsallvl.txt', 'DTwkstat.txt', 'DTsuper.txt', 'DTtoa.txt', 'DTocc.txt', 'DTpp.txt', 'ReadRawData_CreateTempDatasets_DataGov.sas', 'DTedlvl.txt', 'DTloslvl.txt', 'DTstemocc.txt', 'DTloc.txt']


# Data

In [10]:
%%time
df_fact = pd.read_csv('FACTDATA_MAR2024.TXT', low_memory=False)

CPU times: user 2.84 s, sys: 661 ms, total: 3.5 s
Wall time: 3.5 s


In [11]:
df_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2278730 entries, 0 to 2278729
Data columns (total 20 columns):
 #   Column      Dtype  
---  ------      -----  
 0   AGYSUB      object 
 1   LOC         object 
 2   AGELVL      object 
 3   EDLVL       object 
 4   GSEGRD      object 
 5   LOSLVL      object 
 6   OCC         object 
 7   PATCO       int64  
 8   PP          object 
 9   PPGRD       object 
 10  SALLVL      int64  
 11  STEMOCC     object 
 12  SUPERVIS    object 
 13  TOA         object 
 14  WORKSCH     object 
 15  WORKSTAT    int64  
 16  DATECODE    int64  
 17  EMPLOYMENT  int64  
 18  SALARY      float64
 19  LOS         float64
dtypes: float64(2), int64(5), object(13)
memory usage: 347.7+ MB


In [12]:
df_fact.head()

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,SALLVL,STEMOCC,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS
0,AA00,11,F,13,NaN,G,0340,2,ES,ES-**,20,XXXX,2,50,F,1,202403,1,210000.0,20.8
1,AA00,11,J,15,NaN,I,0905,1,ES,ES-**,20,XXXX,2,50,F,1,202403,1,203000.0,31.2
2,AA00,11,K,04,NaN,G,0301,2,99,EX-02,30,XXXX,2,48,F,2,202403,1,NaN,22.0
3,AA00,11,C,04,12,B,0560,2,99,GS-12,9,XXXX,8,15,F,1,202403,1,99200.0,2.5
4,AA00,11,D,04,13,B,0905,1,99,GS-13,11,XXXX,8,30,F,1,202403,1,117962.0,2.5


# Exploratory Analysis

- Geographical distribution
- Number of workers by Agency
- Most common job series
- ...

In [35]:
total_count = df_fact.shape[0]
total_count

2278730

## by Number of Employees

In [14]:
agy_df=pd.read_csv('DTagy.txt')

In [36]:
# AGYTYP,AGYTYPT,AGY,AGYT,AGYSUB,AGYSUBT
agency_freq=df_fact['AGYSUB'].value_counts().to_frame().reset_index().merge(agy_df, on = 'AGYSUB')
agency_freq['proportion'] = agency_freq['count'] / total_count

In [37]:
# Top 10 agencies by number of employees
agency_freq[['AGYSUBT','count','proportion']].head(10)

,AGYSUBT,count,proportion
0,VATA-VETERANS HEALTH ADMINISTRATION,432908,0.189978
1,TR93-INTERNAL REVENUE SERVICE,93937,0.041223
2,AF1M-AIR FORCE MATERIEL COMMAND,70084,0.030756
3,HSBD-CUSTOMS AND BORDER PROTECTION,65268,0.028642
4,HSBC-TRANSPORTATION SECURITY ADMINISTRATION,62788,0.027554
5,SZ00-SOCIAL SECURITY ADMINISTRATION,59227,0.025991
6,DD83-MILITARY TREATMENT FACILITIES UNDER DHA,45754,0.020079
7,TD03-FEDERAL AVIATION ADMINISTRATION,45306,0.019882
8,ARCE-U.S. ARMY CORPS OF ENGINEERS,38079,0.016711
9,DJ02-FEDERAL BUREAU OF INVESTIGATION,37394,0.016410


## by Job Series (Occupation)

In [28]:
occ = pd.read_csv('DTocc.txt')

In [38]:
occ_freq=df_fact[['OCC']].value_counts().to_frame().reset_index().merge(occ, on='OCC')
occ_freq['proportion'] = occ_freq['count'] / total_count

In [39]:
# top 10 occupations
occ_freq[['OCCT','count','proportion']].head(10)

,OCCT,count,proportion
0,0301-MISCELLANEOUS ADMINISTRATION AND PROGRAM,111418,0.048895
1,0610-NURSE,111303,0.048844
2,2210-INFORMATION TECHNOLOGY MANAGEMENT,99051,0.043468
3,0343-MANAGEMENT AND PROGRAM ANALYSIS,94333,0.041397
4,1802-COMPLIANCE INSPECTION AND SUPPORT,55782,0.024479
5,0303-MISCELLANEOUS CLERK AND ASSISTANT,48493,0.021281
6,0905-GENERAL ATTORNEY,44317,0.019448
7,1811-CRIMINAL INVESTIGATION,44171,0.019384
8,1102-CONTRACTING,43549,0.019111
9,0679-MEDICAL SUPPORT ASSISTANCE,42599,0.018694


## by Location

In [41]:
loc = pd.read_csv('DTloc.txt')

In [44]:
loc.head()

,LOCTYP,LOCTYPT,LOC,LOCT
0,1,United States,01,01-ALABAMA
1,1,United States,02,02-ALASKA
2,1,United States,04,04-ARIZONA
3,1,United States,05,05-ARKANSAS
4,1,United States,06,06-CALIFORNIA


In [51]:
loc_freq=df_fact[['LOC']].value_counts().\
to_frame().reset_index().merge(loc[['LOCTYP','LOC','LOCT']], on = 'LOC')
loc_freq['proportion'] = loc_freq['count'] / total_count

In [56]:
loc_freq_us = loc_freq.loc[loc_freq['LOCTYP']==1]

In [58]:
loc_freq_us_50st = loc_freq_us.loc[loc_freq_us['LOC'] != 'US']

In [60]:
loc_freq_us_50st.head()

,LOC,count,LOCTYP,LOCT,proportion
1,11,162144,1,11-DISTRICT OF COLUMBIA,0.071155
2,06,147487,1,06-CALIFORNIA,0.064723
3,51,144483,1,51-VIRGINIA,0.063405
4,24,142876,1,24-MARYLAND,0.062700
5,48,129738,1,48-TEXAS,0.056934
